In [2]:
import pandas as pd
from data_preprocessing import get_cleaned_data
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules


In [4]:
data = get_cleaned_data()

target_value = 0

# select only the columns with target = 0
data = data[data["TARGET"] == target_value]

# convert numbers to categories
for col in data.columns:
    data[col] = data[col].map({x: col + "_" + str(x) for x in data[col].unique()})
        
print(len(data))


282686


In [5]:
split_by_col = 'NAME_EDUCATION_TYPE'    
unique_values = data[split_by_col].unique()
datasets = {}
for value in unique_values:
    if value == "NAME_EDUCATION_TYPE_Higher education":
        continue
    splitted_data = data[data[split_by_col] == value]
    print(f"VA {value} has {len(splitted_data)} rows")
    dataset = create_dataset(splitted_data, None)
    te = TransactionEncoder()
    te_ary = te.fit(dataset).transform(dataset)
    df = pd.DataFrame(te_ary, columns=te.columns_)
    frequent_itemsets = apriori(df, min_support=0.6, use_colnames=True)
    print("Freq itemsets created")
    frequent_itemsets.to_csv(f"../data/{value}_frequent_itemsets_{str(target_value)}.csv")

VA NAME_EDUCATION_TYPE_Secondary / secondary special has 198867 rows
Dataset created


MemoryError: Unable to allocate 31.7 GiB for an array with shape (198867, 171131) and data type bool

In [4]:
frequent_itemsets = pd.read_csv(f"../data/TARGET_1_frequent_itemsets_1.csv")
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7, support_only=True)
print("Rules created")